In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import sys
import os

from skimage.morphology import binary_erosion, binary_dilation, disk
from skimage.measure import label

sys.path.append('/Users/Michael/projects/shape_sharing/src/')
from common import paths, voxel_data, images, parameters

In [4]:
sequence = paths.RenderedData.test_sequence()[10]

vox_dir = paths.RenderedData.ground_truth_voxels(sequence['scene'])
gt_vox = voxel_data.load_voxels(vox_dir)
gt_vox.V[np.isnan(gt_vox.V)] = -parameters.RenderedVoxelGrid.mu
gt_vox.set_origin(gt_vox.origin)

plt.imshow(gt_vox.V[:, :, 10]);

In [25]:
#  really want to just segment in the x-y direction for now...
# make it very simple and quick
gt_vox.segment_project_2d(z_threshold=2, floor_height=4)

plt.imshow(gt_vox.labels[:, :, 10]);
plt.colorbar()

In [14]:
# now need to propgate these labels to the image itself...
frame_data = paths.RenderedData.load_scene_data(
        sequence['scene'], sequence['frames'][0])
im = images.RGBDImage.load_from_dict(
    paths.RenderedData.scene_dir(sequence['scene']),
    frame_data)
plt.imshow(im.depth);

In [24]:
im.label_from_grid(gt_vox)
plt.imshow(im.labels)

In [69]:
plt.figure(figsize=(8, 15))
sp_u, sp_v = 7, 3

for idx, sequence in enumerate(paths.RenderedData.train_sequence()):

    vox_dir = paths.RenderedData.ground_truth_voxels(sequence['scene'])
    gt_vox = voxel_data.load_voxels(vox_dir)
    gt_vox.V[np.isnan(gt_vox.V)] = -parameters.RenderedVoxelGrid.mu
    gt_vox.set_origin(gt_vox.origin)

    gt_vox.segment_project_2d(z_threshold=2, floor_height=4)

    sequence = {'frames': [25, 26, 27, 28, 29], 'name': '27h08gup95ah4us3_SEQ', 'scene': 'O2OOU0UM3XWN_SCN'}
    # now need to propgate these labels to the image itself...
    frame_data = paths.RenderedData.load_scene_data(
            sequence['scene'], sequence['frames'][0])
    im = images.RGBDImage.load_from_dict(
        paths.RenderedData.scene_dir(sequence['scene']),
        frame_data)

    im.label_from_grid(gt_vox)
    plt.subplot(sp_u, sp_v, 3*idx+1)
    plt.imshow(im.depth)
    plt.subplot(sp_u, sp_v, 3*idx+2)
    plt.imshow(gt_vox.temp_2d_labels)
    plt.subplot(sp_u, sp_v, 3*idx+3)
    plt.imshow(im.labels)
    if idx >= (sp_u*sp_v)/3.0 -2 :
        break

In [67]:
# Now am going to attempt to isolate a single label in the grid...
# By dilating the item label
label_to_use = 1
from copy import deepcopy

mu = parameters.RenderedVoxelGrid.mu
extracted_V = deepcopy(gt_vox.V)
extracted_V[gt_vox.labels !=label_to_use] = mu

plt.figure(figsize=(10, 5))
plt.subplot(131)
plt.imshow(gt_vox.V[:, :, 10])
plt.clim((-mu, mu))
plt.subplot(132)
plt.imshow(gt_vox.labels[:, :, 10] == label_to_use)
plt.subplot(133)
plt.imshow(extracted_V[:, :, 10])
plt.clim((-mu, mu))

In [68]:
for f in paths.RenderedData.train_sequence():
    print f
#27h08gup95ah4us3_SEQ